# Valley Walking Algorithm

Complete implementation: valley detection → adaptive walking → visualization.

In [22]:
# Globtim Notebook Setup - Universal Header Cell
# This cell automatically detects your environment and sets up the appropriate configuration
# No editing required - works from any location in the project

include(joinpath(dirname(Base.find_package("Globtim")), "..", ".globtim", "notebook_setup.jl"))

Environment detected: local
Setting up local development environment...
Loading CairoMakie...
CairoMakie activated for high-quality plots
GLMakie available for interactive plots
Loading Globtim from main project...
Globtim loaded successfully!
Ready for local development!
Available: Full plotting, interactive development tools
Switch plotting: GLMakie.activate!() for interactive plots


  Activating project at `~/globtim/environments/local`


In [23]:
# Load required packages
using Globtim
using LinearAlgebra
using ForwardDiff
using DataFrames
using DynamicPolynomials
using Printf
using CairoMakie

# Activate CairoMakie backend for high-quality static plots
CairoMakie.activate!()
println("✓ CairoMakie backend activated for high-quality static visualization")

✓ CairoMakie backend activated for high-quality static visualization


## Setup

In [24]:
# Test function: unit circle valley
f(x) = (x[1]^2 + x[2]^2 - 1)^2

# Configuration
config = (
    gradient_tolerance = 1e-4,
    eigenvalue_threshold = 1e-6,
    initial_step_size = 1e-3,
    max_steps = 150,
    function_tolerance = 1e-8
)

(gradient_tolerance = 0.0001, eigenvalue_threshold = 1.0e-6, initial_step_size = 0.001, max_steps = 150, function_tolerance = 1.0e-8)

## Valley Detection

In [25]:
function detect_valley(f, point, config)
    grad = ForwardDiff.gradient(f, point)
    hess = ForwardDiff.hessian(f, point)
    eigenvals = eigvals(hess)
    
    grad_norm = norm(grad)
    valley_dimension = sum(abs.(eigenvals) .< config.eigenvalue_threshold)
    
    is_critical = grad_norm < config.gradient_tolerance
    is_valley = is_critical && (valley_dimension > 0)
    
    if is_valley
        eigendecomp = eigen(hess)
        valley_mask = abs.(eigendecomp.values) .< config.eigenvalue_threshold
        valley_directions = eigendecomp.vectors[:, valley_mask]
        return true, valley_directions
    end
    return false, nothing
end

# Test on unit circle points
test_points = [[1.0, 0.0], [0.0, 1.0], [-1.0, 0.0], [0.707, 0.707]]
valley_points = []
all_directions = []

for point in test_points
    is_valley, directions = detect_valley(f, point, config)
    if is_valley
        push!(valley_points, point)
        push!(all_directions, directions)
        println("✓ Valley at $(point)")
    end
end

✓ Valley at [1.0, 0.0]
✓ Valley at [0.0, 1.0]
✓ Valley at [-1.0, 0.0]


## Adaptive Valley Walking

In [26]:
function adaptive_valley_walk_with_momentum(f, start_point, direction, config)
    steps = [start_point]
    current_point = copy(start_point)
    current_step_size = config.initial_step_size
    direction = direction / norm(direction)
    
    # Nesterov-style momentum parameters
    momentum = 0.9  # momentum coefficient
    velocity = zeros(length(start_point))  # velocity vector
    distance_bonus = 0.1  # incentive to move away from start
    
    for step_num in 1:config.max_steps
        # Nesterov look-ahead: compute gradient at projected position
        lookahead_point = current_point + momentum * velocity
        lookahead_grad = ForwardDiff.gradient(f, lookahead_point)
        
        # Compute distance from start point (incentive to explore)
        distance_from_start = norm(current_point - start_point)
        distance_incentive = distance_bonus / (1.0 + distance_from_start)
        
        # Adaptive direction: combine valley direction with momentum
        effective_direction = direction + 0.1 * velocity / max(norm(velocity), 1e-12)
        effective_direction = effective_direction / norm(effective_direction)
        
        # Candidate step with momentum
        candidate_point = current_point + current_step_size * effective_direction
        candidate_f = f(candidate_point)
        current_f = f(current_point)
        
        # Validation with momentum consideration
        grad = ForwardDiff.gradient(f, candidate_point)
        grad_norm = norm(grad)
        
        # Enhanced acceptance criteria with distance incentive
        f_change = candidate_f - current_f
        acceptable_increase = config.function_tolerance + distance_incentive
        
        if f_change <= acceptable_increase && grad_norm <= config.gradient_tolerance
            # Accept step and update momentum
            step_vector = candidate_point - current_point
            velocity = momentum * velocity + (1 - momentum) * step_vector
            
            current_point = candidate_point
            push!(steps, copy(candidate_point))
            
            # Increase step size for efficiency
            current_step_size = min(current_step_size * 1.2, 1e-2)
        else
            # Reduce step size but maintain some momentum
            current_step_size = max(current_step_size * 0.5, 1e-12)
            velocity *= 0.9  # decay velocity on failed steps
            
            if current_step_size < 1e-12
                break
            end
        end
    end
    return steps
end

# Walk from each valley point with momentum
all_paths = Dict{String, Vector{Vector{Float64}}}()

for (i, (point, directions)) in enumerate(zip(valley_points, all_directions))
    for (j, direction) in enumerate(eachcol(directions))
        pos_path = adaptive_valley_walk_with_momentum(f, point, direction, config)
        neg_path = adaptive_valley_walk_with_momentum(f, point, -direction, config)
        
        all_paths["point_$(i)_dir_$(j)_pos"] = pos_path
        all_paths["point_$(i)_dir_$(j)_neg"] = neg_path
    end
end

println("Total paths with momentum: $(length(all_paths))")
for (name, path) in all_paths
    println("  $(name): $(length(path)) points")
end

Total paths with momentum: 6
  point_2_dir_1_neg: 20 points
  point_2_dir_1_pos: 20 points
  point_3_dir_1_pos: 20 points
  point_1_dir_1_pos: 20 points
  point_1_dir_1_neg: 20 points
  point_3_dir_1_neg: 20 points


## Visualization

In [27]:
# Create visualization
GLMakie.activate!()  # Switch to interactive backend
fig = Figure(size = (1000, 800))
ax = Axis(fig[1, 1], xlabel = "x₁", ylabel = "x₂", title = "Valley Walking Results", aspect = DataAspect())

# Contour plot
x_range = range(-2.2, 2.2, length=200)
y_range = range(-2.2, 2.2, length=200)
Z = [f([x, y]) for y in y_range, x in x_range]
levels = [0.0, 0.01, 0.05, 0.1, 0.25, 0.5, 1.0, 2.0, 5.0]
contourf!(ax, x_range, y_range, Z, levels = levels, colormap = :viridis)
contour!(ax, x_range, y_range, Z, levels = levels, color = :black, linewidth = 0.5)

# Valley start points
for point in valley_points
    scatter!(ax, [point[1]], [point[2]], color = :red, markersize = 25, marker = :star5)
end

# Valley paths with labels
colors = [:lime, :blue, :orange, :purple, :cyan, :magenta]
path_count = 0

for (path_name, path_points) in all_paths
    if length(path_points) > 1
        path_count += 1
        color = colors[mod1(path_count, length(colors))]
        
        x_coords = [p[1] for p in path_points]
        y_coords = [p[2] for p in path_points]
        
        lines!(ax, x_coords, y_coords, color = color, linewidth = 3, label = path_name)
        scatter!(ax, x_coords, y_coords, color = color, markersize = 6)
        
        # Mark start and end
        scatter!(ax, [x_coords[1]], [y_coords[1]], color = color, markersize = 12, marker = :circle)
        scatter!(ax, [x_coords[end]], [y_coords[end]], color = color, markersize = 12, marker = :rect)
    end
end

axislegend(ax, position = :lt, framevisible = true, backgroundcolor = (:white, 0.8))
xlims!(ax, -2.2, 2.2)
ylims!(ax, -2.2, 2.2)

# Display interactive plot
display(fig)

# Also save static version
CairoMakie.activate!()
fig_static = Figure(size = (1000, 800))
ax_static = Axis(fig_static[1, 1], xlabel = "x₁", ylabel = "x₂", title = "Valley Walking Results", aspect = DataAspect())

contourf!(ax_static, x_range, y_range, Z, levels = levels, colormap = :viridis)
contour!(ax_static, x_range, y_range, Z, levels = levels, color = :black, linewidth = 0.5)

for point in valley_points
    scatter!(ax_static, [point[1]], [point[2]], color = :red, markersize = 25, marker = :star5)
end

path_count = 0
for (path_name, path_points) in all_paths
    if length(path_points) > 1
        path_count += 1
        color = colors[mod1(path_count, length(colors))]
        
        x_coords = [p[1] for p in path_points]
        y_coords = [p[2] for p in path_points]
        
        lines!(ax_static, x_coords, y_coords, color = color, linewidth = 3, label = path_name)
        scatter!(ax_static, x_coords, y_coords, color = color, markersize = 6)
        
        scatter!(ax_static, [x_coords[1]], [y_coords[1]], color = color, markersize = 12, marker = :circle)
        scatter!(ax_static, [x_coords[end]], [y_coords[end]], color = color, markersize = 12, marker = :rect)
    end
end

axislegend(ax_static, position = :lt, framevisible = true, backgroundcolor = (:white, 0.8))
xlims!(ax_static, -2.2, 2.2)
ylims!(ax_static, -2.2, 2.2)

filename = "valley_walking_results.png"
save(filename, fig_static, px_per_unit = 3)
println("Saved static version: $(filename)")

# Switch back to GLMakie for interactive display
GLMakie.activate!()
fig

Saved static version: valley_walking_results.png


## Summary

Valley walking algorithm:
1. **Detection**: Hessian eigenanalysis identifies valley directions
2. **Walking**: Adaptive step sizing with gradient validation
3. **Visualization**: High-quality static plots with CairoMakie

In [28]:
# Results
total_points = sum(length.(values(all_paths)))
println("Valley walking complete:")
println("  Paths: $(length(all_paths))")
println("  Total points: $(total_points)")
println("  Output: $(filename)")

Valley walking complete:
  Paths: 6
  Total points: 120
  Output: valley_walking_results.png
